In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

from pathlib import Path
import re

from IPython.core.display import display_latex

from visu_utils import TASK_LIST_CLASSIFICATION, make_results_tables_per_group_size, load_classification_merged_mteb, \
    load_sts_merged_mteb

from autorank import autorank, plot_stats, create_report, latex_table

EXPORT_PATH_TABLE = Path("../../papers/Distillation-MI-ICLR/tables/nlp/")
EXPORT_PATH_FIG = Path("../../papers/Distillation-MI-ICLR/figures/nlp/")

# Make sure the folders exist
EXPORT_PATH_TABLE.mkdir(parents=True, exist_ok=True)
EXPORT_PATH_FIG.mkdir(parents=True, exist_ok=True)

%reload_ext autoreload
%autoreload 2


In [27]:

MTEB_BASELINES_PATH = Path("../non_sync/baselines_mteb/mteb_detailed/en_STS.csv")

RESULTS_PATHS = [Path("../non_sync/mteb_benchmarking/results/experiments_gist_nll"),
                 Path("../non_sync/mteb_benchmarking/results/experiments_gist_mse")
                 ]



df_merged = load_sts_merged_mteb(MTEB_BASELINES_PATH, RESULTS_PATHS)


# select only last training step for each model
df_merged = df_merged.reset_index()

indices = df_merged.sort_values('Training step').groupby('Model')['Training step'].idxmax()
df_merged = df_merged.loc[indices]
df_merged = df_merged.drop('Training step', axis=1)


df_melted = df_merged.melt(id_vars=['Model', 'Model Size (Million Parameters)', 'loss', 'Dataset'],
                           value_vars=df_merged.columns[4:], var_name='Task', value_name='Score')


df_melted = df_melted[~df_melted['Task'].str.contains('SummEval')]


In [28]:

df_melted

,Model,Model Size (Million Parameters),loss,Dataset,Task,Score
0,Alibaba-NLP/gte-Qwen1.5-7B-instruct,7099.0,MTEB,MTEB,BIOSSES,81.12
1,Alibaba-NLP/gte-Qwen2-1.5B-instruct,1776.0,MTEB,MTEB,BIOSSES,82.11
2,Alibaba-NLP/gte-Qwen2-7B-instruct,7613.0,MTEB,MTEB,BIOSSES,81.37
3,Alibaba-NLP/gte-base-en-v1.5,137.0,MTEB,MTEB,BIOSSES,83.65
4,Alibaba-NLP/gte-large-en-v1.5,434.0,MTEB,MTEB,BIOSSES,85.39
...,...,...,...,...,...,...
1375,thenlper/gte-small,33.0,MTEB,MTEB,STSBenchmark,85.57
1376,thtang/ALL_862873,118.0,MTEB,MTEB,STSBenchmark,44.39
1377,w601sxs/b1ade-embed,335.0,MTEB,MTEB,STSBenchmark,88.77
1378,zeta-alpha-ai/Zeta-Alpha-E5-Mistral,7111.0,MTEB,MTEB,STSBenchmark,88.28


In [29]:
# to downsample

models = ['paraphrase-multilingual', 'msmarco', 'ALL', "m-v1.5", "unsup-sim"]


In [30]:

size_ranges = [(16, 30), (30, 50), (100, 120)]

idx = pd.IndexSlice
for k, (low, high) in enumerate(size_ranges):
    latex_results = df_melted[
        (df_melted['Model Size (Million Parameters)'] >= low) & (
                df_melted['Model Size (Million Parameters)'] <= high)].copy()

    
    latex_results = latex_results.dropna()
    latex_results['Task'] = latex_results['Task'].apply(lambda x: re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', x))
    latex_results['Task'] = latex_results['Task'].apply(lambda x: x.replace(" (en)", ""))
    latex_results['Task'] = latex_results['Task'].apply(lambda x: x.replace(" ", " \\\\ "))
    latex_results['Task'] = latex_results['Task'].apply(lambda x: f"\\rotatebox{{90}}{{\\shortstack{{{x}}}}}")

    make_results_tables_per_group_size(latex_results, "Score", low, high, EXPORT_PATH_TABLE,
                                       f"mteb_sts_per_size_{low}_{high}",
                                       caption=f"Performance of our distilled models compared of models of similar sizes {low}M to {high}M parameters from the MTEB Benchmark on STS tasks.")

    for model in models:
        latex_results = latex_results[~latex_results['Model'].str.contains(model)]

    make_results_tables_per_group_size(latex_results, "Score", low, high, EXPORT_PATH_TABLE,
                                       name="mteb_sts_per_size_{low}_{high}_downsampled",
                                       caption=f"Performance of our distilled models compared of models of similar sizes {low}M to {high}M parameters from the MTEB Benchmark on STS tasks.")








Task,,Model,Params. (M),\rotatebox{90}{\shortstack{BIOSSES}},\rotatebox{90}{\shortstack{SICK-R}},\rotatebox{90}{\shortstack{STS12}},\rotatebox{90}{\shortstack{STS13}},\rotatebox{90}{\shortstack{STS14}},\rotatebox{90}{\shortstack{STS15}},\rotatebox{90}{\shortstack{STS16}},\rotatebox{90}{\shortstack{STS17}},\rotatebox{90}{\shortstack{STS22}},\rotatebox{90}{\shortstack{STSBenchmark}}
0,MSE,MSE/GIST/Student-xs,23.0,76.828859,78.898160,71.375237,78.873817,75.627732,84.593771,82.297623,86.822035,66.693038,82.616256
1,MTEB,Mihaiii/Bulbasaur,17.0,85.050000,76.040000,69.500000,80.960000,77.080000,85.420000,82.300000,88.030000,64.120000,83.340000
2,MTEB,Mihaiii/Ivysaur,23.0,87.320000,75.630000,68.580000,80.540000,77.630000,86.160000,82.820000,88.570000,67.390000,84.250000
3,MTEB,Mihaiii/Squirtle,16.0,71.780000,77.340000,70.150000,78.420000,74.760000,82.000000,78.270000,85.850000,61.200000,79.210000
4,MTEB,Mihaiii/Venusaur,16.0,77.590000,74.680000,54.350000,74.240000,69.990000,75.740000,73.650000,84.810000,62.560000,76.720000
5,MTEB,Mihaiii/Wartortle,17.0,80.780000,78.240000,75.190000,79.330000,76.560000,84.700000,81.440000,86.610000,63.430000,81.790000
6,MTEB,Snowflake/snowflake-arctic-embed-xs,23.0,84.050000,69.260000,65.900000,77.870000,72.820000,83.490000,80.580000,84.490000,66.280000,79.180000
7,MTEB,TaylorAI/bge-micro,17.0,83.420000,72.390000,71.900000,80.930000,76.600000,84.920000,80.720000,85.610000,65.900000,81.320000
8,MTEB,TaylorAI/bge-micro-v2,17.0,82.920000,73.620000,71.890000,79.850000,76.860000,84.770000,81.910000,86.820000,65.380000,82.500000
9,MTEB,TaylorAI/gte-tiny,23.0,86.630000,75.850000,72.580000,82.390000,77.980000,86.540000,83.310000,88.280000,66.680000,84.380000


Task,,Model,Params. (M),\rotatebox{90}{\shortstack{BIOSSES}},\rotatebox{90}{\shortstack{SICK-R}},\rotatebox{90}{\shortstack{STS12}},\rotatebox{90}{\shortstack{STS13}},\rotatebox{90}{\shortstack{STS14}},\rotatebox{90}{\shortstack{STS15}},\rotatebox{90}{\shortstack{STS16}},\rotatebox{90}{\shortstack{STS17}},\rotatebox{90}{\shortstack{STS22}},\rotatebox{90}{\shortstack{STSBenchmark}}
0,MSE,MSE/GIST/Student-xs,23.0,76.828859,78.898160,71.375237,78.873817,75.627732,84.593771,82.297623,86.822035,66.693038,82.616256
1,MTEB,Mihaiii/Bulbasaur,17.0,85.050000,76.040000,69.500000,80.960000,77.080000,85.420000,82.300000,88.030000,64.120000,83.340000
2,MTEB,Mihaiii/Ivysaur,23.0,87.320000,75.630000,68.580000,80.540000,77.630000,86.160000,82.820000,88.570000,67.390000,84.250000
3,MTEB,Mihaiii/Squirtle,16.0,71.780000,77.340000,70.150000,78.420000,74.760000,82.000000,78.270000,85.850000,61.200000,79.210000
4,MTEB,Mihaiii/Venusaur,16.0,77.590000,74.680000,54.350000,74.240000,69.990000,75.740000,73.650000,84.810000,62.560000,76.720000
5,MTEB,Mihaiii/Wartortle,17.0,80.780000,78.240000,75.190000,79.330000,76.560000,84.700000,81.440000,86.610000,63.430000,81.790000
6,MTEB,Snowflake/snowflake-arctic-embed-xs,23.0,84.050000,69.260000,65.900000,77.870000,72.820000,83.490000,80.580000,84.490000,66.280000,79.180000
7,MTEB,TaylorAI/bge-micro,17.0,83.420000,72.390000,71.900000,80.930000,76.600000,84.920000,80.720000,85.610000,65.900000,81.320000
8,MTEB,TaylorAI/bge-micro-v2,17.0,82.920000,73.620000,71.890000,79.850000,76.860000,84.770000,81.910000,86.820000,65.380000,82.500000
9,MTEB,TaylorAI/gte-tiny,23.0,86.630000,75.850000,72.580000,82.390000,77.980000,86.540000,83.310000,88.280000,66.680000,84.380000


Task,,Model,Params. (M),\rotatebox{90}{\shortstack{BIOSSES}},\rotatebox{90}{\shortstack{SICK-R}},\rotatebox{90}{\shortstack{STS12}},\rotatebox{90}{\shortstack{STS13}},\rotatebox{90}{\shortstack{STS14}},\rotatebox{90}{\shortstack{STS15}},\rotatebox{90}{\shortstack{STS16}},\rotatebox{90}{\shortstack{STS17}},\rotatebox{90}{\shortstack{STS22}},\rotatebox{90}{\shortstack{STSBenchmark}}
0,MSE,MSE/GIST/Student-s,33.0,74.840862,78.931793,70.055913,78.196301,74.090338,83.242073,81.498496,85.633134,65.860047,82.295341
1,MTEB,BAAI/bge-small-en-v1.5,33.0,83.750000,79.410000,77.440000,82.980000,81.840000,87.260000,84.930000,87.150000,65.300000,85.860000
2,MTEB,Snowflake/snowflake-arctic-embed-s,33.0,86.270000,69.660000,68.790000,79.620000,75.580000,84.640000,82.400000,86.730000,69.490000,81.190000
3,MTEB,andersonbcdefg/bge-small-4096,35.0,81.600000,74.180000,72.200000,80.540000,76.200000,85.200000,81.930000,86.610000,65.460000,81.940000
4,MTEB,avsolatorio/GIST-small-Embedding-v0,33.0,86.990000,80.530000,75.570000,86.260000,82.300000,88.740000,85.270000,89.020000,68.510000,87.080000
5,MTEB,avsolatorio/NoInstruct-small-Embedding-v0,33.0,87.200000,80.310000,75.760000,86.080000,82.280000,88.900000,85.180000,88.730000,68.540000,86.980000
6,MTEB,intfloat/e5-small,33.0,84.220000,78.900000,75.190000,81.800000,78.480000,87.490000,84.580000,87.940000,63.760000,86.360000
7,MTEB,intfloat/e5-small-v2,33.0,79.430000,78.510000,76.210000,82.400000,79.000000,87.760000,83.800000,87.720000,63.150000,85.950000
8,MTEB,jinaai/jina-embedding-s-en-v1,35.0,82.960000,76.330000,74.280000,78.550000,73.840000,83.710000,80.030000,87.490000,64.250000,79.200000
9,MTEB,jinaai/jina-embeddings-v2-small-en,33.0,80.520000,76.720000,73.660000,83.300000,79.170000,87.300000,83.600000,88.230000,63.460000,84.040000


Task,,Model,Params. (M),\rotatebox{90}{\shortstack{BIOSSES}},\rotatebox{90}{\shortstack{SICK-R}},\rotatebox{90}{\shortstack{STS12}},\rotatebox{90}{\shortstack{STS13}},\rotatebox{90}{\shortstack{STS14}},\rotatebox{90}{\shortstack{STS15}},\rotatebox{90}{\shortstack{STS16}},\rotatebox{90}{\shortstack{STS17}},\rotatebox{90}{\shortstack{STS22}},\rotatebox{90}{\shortstack{STSBenchmark}}
0,MSE,MSE/GIST/Student-s,33.0,74.840862,78.931793,70.055913,78.196301,74.090338,83.242073,81.498496,85.633134,65.860047,82.295341
1,MTEB,BAAI/bge-small-en-v1.5,33.0,83.750000,79.410000,77.440000,82.980000,81.840000,87.260000,84.930000,87.150000,65.300000,85.860000
2,MTEB,Snowflake/snowflake-arctic-embed-s,33.0,86.270000,69.660000,68.790000,79.620000,75.580000,84.640000,82.400000,86.730000,69.490000,81.190000
3,MTEB,andersonbcdefg/bge-small-4096,35.0,81.600000,74.180000,72.200000,80.540000,76.200000,85.200000,81.930000,86.610000,65.460000,81.940000
4,MTEB,avsolatorio/GIST-small-Embedding-v0,33.0,86.990000,80.530000,75.570000,86.260000,82.300000,88.740000,85.270000,89.020000,68.510000,87.080000
5,MTEB,avsolatorio/NoInstruct-small-Embedding-v0,33.0,87.200000,80.310000,75.760000,86.080000,82.280000,88.900000,85.180000,88.730000,68.540000,86.980000
6,MTEB,intfloat/e5-small,33.0,84.220000,78.900000,75.190000,81.800000,78.480000,87.490000,84.580000,87.940000,63.760000,86.360000
7,MTEB,intfloat/e5-small-v2,33.0,79.430000,78.510000,76.210000,82.400000,79.000000,87.760000,83.800000,87.720000,63.150000,85.950000
8,MTEB,jinaai/jina-embedding-s-en-v1,35.0,82.960000,76.330000,74.280000,78.550000,73.840000,83.710000,80.030000,87.490000,64.250000,79.200000
9,MTEB,jinaai/jina-embeddings-v2-small-en,33.0,80.520000,76.720000,73.660000,83.300000,79.170000,87.300000,83.600000,88.230000,63.460000,84.040000


Task,,Model,Params. (M),\rotatebox{90}{\shortstack{BIOSSES}},\rotatebox{90}{\shortstack{SICK-R}},\rotatebox{90}{\shortstack{STS12}},\rotatebox{90}{\shortstack{STS13}},\rotatebox{90}{\shortstack{STS14}},\rotatebox{90}{\shortstack{STS15}},\rotatebox{90}{\shortstack{STS16}},\rotatebox{90}{\shortstack{STS17}},\rotatebox{90}{\shortstack{STS22}},\rotatebox{90}{\shortstack{STSBenchmark}}
0,MSE,MSE/GIST/Student-m,109.0,82.676583,81.068982,73.496009,81.949051,78.168532,86.333299,84.653138,88.051412,66.003694,85.063858
1,MTEB,BAAI/bge-base-en-v1.5,109.0,86.940000,80.300000,78.030000,84.190000,82.270000,87.960000,85.480000,86.420000,65.950000,86.420000
2,MTEB,Lajavaness/bilingual-embedding-small,118.0,83.990000,74.650000,79.390000,85.320000,83.900000,88.470000,84.440000,85.770000,67.240000,86.080000
3,MTEB,Marqo/multilingual-e5-small,118.0,82.260000,77.510000,76.560000,76.970000,75.520000,87.120000,83.630000,86.440000,60.940000,84.010000
4,MTEB,Snowflake/snowflake-arctic-embed-m,109.0,86.620000,69.120000,66.970000,79.120000,68.510000,79.920000,78.660000,81.460000,65.840000,74.100000
5,MTEB,Snowflake/snowflake-arctic-embed-m-v1.5,109.0,86.400000,69.860000,61.800000,82.670000,68.950000,75.550000,77.270000,74.970000,69.080000,69.720000
6,MTEB,avsolatorio/GIST-Embedding-v0,109.0,87.950000,81.290000,76.160000,87.850000,83.390000,89.430000,85.350000,88.590000,67.810000,87.320000
7,MTEB,current/ml-nlp-elser.html,110.0,83.790000,68.780000,64.810000,80.100000,74.960000,83.700000,80.550000,85.740000,67.500000,79.540000
8,MTEB,dwzhu/e5-base-4k,112.0,81.400000,78.300000,75.790000,83.580000,79.950000,88.820000,84.460000,87.580000,64.070000,86.520000
9,MTEB,hkunlp/instructor-base,110.0,82.310000,80.260000,77.020000,86.580000,81.320000,88.190000,84.880000,89.460000,66.450000,86.430000


Task,,Model,Params. (M),\rotatebox{90}{\shortstack{BIOSSES}},\rotatebox{90}{\shortstack{SICK-R}},\rotatebox{90}{\shortstack{STS12}},\rotatebox{90}{\shortstack{STS13}},\rotatebox{90}{\shortstack{STS14}},\rotatebox{90}{\shortstack{STS15}},\rotatebox{90}{\shortstack{STS16}},\rotatebox{90}{\shortstack{STS17}},\rotatebox{90}{\shortstack{STS22}},\rotatebox{90}{\shortstack{STSBenchmark}}
0,MSE,MSE/GIST/Student-m,109.0,82.676583,81.068982,73.496009,81.949051,78.168532,86.333299,84.653138,88.051412,66.003694,85.063858
1,MTEB,BAAI/bge-base-en-v1.5,109.0,86.940000,80.300000,78.030000,84.190000,82.270000,87.960000,85.480000,86.420000,65.950000,86.420000
2,MTEB,Lajavaness/bilingual-embedding-small,118.0,83.990000,74.650000,79.390000,85.320000,83.900000,88.470000,84.440000,85.770000,67.240000,86.080000
3,MTEB,Marqo/multilingual-e5-small,118.0,82.260000,77.510000,76.560000,76.970000,75.520000,87.120000,83.630000,86.440000,60.940000,84.010000
4,MTEB,Snowflake/snowflake-arctic-embed-m,109.0,86.620000,69.120000,66.970000,79.120000,68.510000,79.920000,78.660000,81.460000,65.840000,74.100000
5,MTEB,avsolatorio/GIST-Embedding-v0,109.0,87.950000,81.290000,76.160000,87.850000,83.390000,89.430000,85.350000,88.590000,67.810000,87.320000
6,MTEB,current/ml-nlp-elser.html,110.0,83.790000,68.780000,64.810000,80.100000,74.960000,83.700000,80.550000,85.740000,67.500000,79.540000
7,MTEB,dwzhu/e5-base-4k,112.0,81.400000,78.300000,75.790000,83.580000,79.950000,88.820000,84.460000,87.580000,64.070000,86.520000
8,MTEB,hkunlp/instructor-base,110.0,82.310000,80.260000,77.020000,86.580000,81.320000,88.190000,84.880000,89.460000,66.450000,86.430000
9,MTEB,huggingface.co/bert-base-uncased,110.0,54.700000,58.650000,30.870000,59.890000,47.730000,60.290000,63.730000,64.100000,56.370000,47.290000
